In [148]:
import utils.video
import importlib
importlib.reload(utils.video)
from utils.video import Video, filter_msgs_to_fps
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from plot_helpers.colors import get_colors, transform_to_cv2
import utils.rosbag
importlib.reload(utils.rosbag)
from utils.rosbag import create_frenet_converter, get_trackbounds_from_bag, get_detections_from_bag, get_states_from_bag, get_opponents_trajectories, get_markers_before

In [ ]:
video_name="/home/moe/ros_eval/videos/mspliner.mp4"
bag="/home/moe/data/multi/video.bag"
HEIGHT=1080
WIDTH=960
FPS=30
colors = [transform_to_cv2(c) for c in get_colors()]
TRACK_COLOR = (200, 200, 200)
OUTSIDE_COLOR = (255, 255 ,255)

In [3]:
ego_states = get_states_from_bag(bag)
ego_states = filter_msgs_to_fps(ego_states, FPS)
fc  = create_frenet_converter(bag)
boundary_ext, boundary_int = get_trackbounds_from_bag(bag)
det = get_detections_from_bag(bag, fc)
d = pd.concat(det, ignore_index=True)
ego_states = filter_msgs_to_fps(ego_states, FPS)
trajectories = get_opponents_trajectories(bag, last_only=False)

Getting states |████████████████████████████████████████| 7738/7738 [100%] in 0.3s (23433.16/s) 
Trackbounds |████████████████████████████████████████| 1119/1119 [100%] in 0.0s (866266.69/s) 
Detections |████████████████████████████████████████| 3087/3087 [100%] in 0.2s (13028.82/s) 


In [4]:
def find_closest_row(df, target_value):
    closest_index = (df["time"] - target_value).abs().idxmin()
    return df.loc[closest_index]

In [104]:
def create_trajectory_frame(frame, df, video, opp_color):
    outer = pd.DataFrame()
    inner = pd.DataFrame()
    outer["x"], outer["y"] = fc.get_cartesian(df["s"], df["d"] + np.sqrt(df["var"]))
    inner["x"], inner["y"] = fc.get_cartesian(df["s"], df["d"] - np.sqrt(df["var"]))
    x, y = fc.get_cartesian(df["s"], df["d"])
    last_frame = video.add_polygons(
        frame, [boundary_ext, outer, inner, boundary_int], colors=[TRACK_COLOR, opp_color, TRACK_COLOR, OUTSIDE_COLOR],alphas=[1.0, 0.6, 1.0, 1.0])
    last_frame = video.add_line(last_frame, x, y, opp_color)
    last_frame = video.add_points(last_frame, old_x, old_y, color=opp_color, alpha=0.6)
    return last_frame, video
    

In [105]:
def add_ot_frames(frame, ot_traj, video, opp_traj, opp_color, num_frames=40):
    x, y = fc.get_cartesian(opp_traj["s"], opp_traj["d"])
    n = ot_traj.shape[0]
    step_frame = video.add_line(frame.copy(), x, y, opp_color)
    for step in range(1, num_frames + 1):
        num_rows = step * n // num_frames
        subset = ot_traj.iloc[:num_rows]
        step_frame = video.add_line(step_frame, subset['x'].to_list(), subset['y'].to_list(), color=colors[0])
        video.add_frame(step_frame)
    return video

In [122]:
START_TIME=7
TRAJ_UPDATE_TIMES = {
    20: 17,
    23: 21,
    25: "OT",
    29: "END_OT",
    40: 31,
    45: 32,
    50: 35,
    56: "OT",
    60: "END_OT",
    60.1: 39,
    65: "OT",
    67: "END_OT"
}


update_iterator = iter(TRAJ_UPDATE_TIMES.keys())
next_update = next(update_iterator)
trajectory_index = None

first_time = ego_states.loc[0]["time"]
vid = Video(video_name, WIDTH, HEIGHT, FPS)
frame = vid.get_empty_frame()
frame = vid.add_polygons(frame, [boundary_ext.copy(), boundary_int.copy()], colors=[TRACK_COLOR, OUTSIDE_COLOR])
old_x = []
old_y = []
ego_traj = None
opp_traj = None
opp_color = colors[1]
opp_index = 0

for idx, state in ego_states.iterrows():
    t = state["time"] - first_time
    if t < START_TIME:
        continue

    detection = find_closest_row(d, state["time"])
    new_frame = frame.copy()

    # add ego car and opponent
    new_frame = vid.add_point(new_frame, detection["x"], detection["y"], color=opp_color)
    new_frame = vid.add_point(new_frame, state["position_x"], state["position_y"], color=colors[0])
    
    # Check if update is happening
    if t > next_update:
        trajectory_index = TRAJ_UPDATE_TIMES[next_update]
        try:
            next_update = next(update_iterator)
        except:
            # traj_frame, vid = create_trajectory_frame(
            #     new_frame, trajectories[trajectory_index][0], vid, opp_color)
            # vid.add_frame(traj_frame, flip=True)
            break
        

    if trajectory_index is not None and trajectory_index not in ["OT", "ACTIVE_OT", "END_OT"]:
        # Draw Detections and Trajectory
        opp_traj = trajectories[trajectory_index][opp_index]
        new_frame = vid.add_points(new_frame, old_x, old_y, color=opp_color, alpha=0.6)
        traj_frame, vid = create_trajectory_frame(
            new_frame, opp_traj, vid, opp_color)
        vid.add_frame(traj_frame, flip=True)
        old_x.pop(0)
        old_y.pop(0)
    elif trajectory_index == "OT":
        # Draw Overtake Trajectory and start overtake
        print("Adding Overtake")
        trajectory_index = "ACTIVE_OT"
        ego_traj = get_markers_before(bag, "/planner/avoidance/markers_sqp", state["time"]) # somehow deletes video?
        ego_traj = ego_traj[:100]
        vid = add_ot_frames(new_frame, ego_traj, vid, opp_traj, opp_color)
    elif trajectory_index == "ACTIVE_OT":
        # Active OT only draw Opponent and Ego car
        new_frame = video.add_line(new_frame, ego_traj['x'].to_list(), ego_traj['y'].to_list(), color=colors[0])
        vid.add_frame(new_frame)
    elif trajectory_index == "END_OT":
        # OT Finish Go back to trail and draw only detections
        trajectory_index = None
        opp_color=colors[2]
        old_x = []
        old_y = []
        opp_index = 1
    elif trajectory_index is None:
        # No trajectory yet only draw detections
        new_frame = vid.add_points(new_frame, old_x, old_y, color=opp_color, alpha=0.6)
        vid.add_frame(new_frame, flip=True)

    old_x.append(detection["x"])
    old_y.append(detection["y"])

vid.finish()

Adding Overtake
Adding Overtake
Adding Overtake
Writing video to /home/moe/ros_eval/videos/mpspliner.mp4
